# Import Library 

In [1]:
import os
GPU = f'0'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
from glob import glob
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from tensorflow.keras.applications import (
    MobileNet,
    MobileNetV2
)
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,
    Dense
)
from sklearn.model_selection import train_test_split

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
batch_size = 1

classes = 1049
size = 224

#### Load Data 

In [4]:
df = pd.read_csv('test.csv')
df.head()

,id
0,../../Datasets/public/test/x/xlf1tgh2ih.JPG
1,../../Datasets/public/test/6/68a3ot4osk.JPG
2,../../Datasets/public/test/s/si2lek4u0a.JPG
3,../../Datasets/public/test/r/rmtqxhipnv.JPG
4,../../Datasets/public/test/2/2flmjdud0e.JPG


In [5]:
df = pd.concat([df, pd.DataFrame([0]*len(df))], axis=1)
df[0] = df[0].astype(str)
df.head()

,id,0
0,../../Datasets/public/test/x/xlf1tgh2ih.JPG,0
1,../../Datasets/public/test/6/68a3ot4osk.JPG,0
2,../../Datasets/public/test/s/si2lek4u0a.JPG,0
3,../../Datasets/public/test/r/rmtqxhipnv.JPG,0
4,../../Datasets/public/test/2/2flmjdud0e.JPG,0


# Datagenerator 

In [6]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_generator = test_datagen.flow_from_dataframe(
    df,
    x_col='id',
    y_col=0,
    target_size=(size,size),
    batch_size=batch_size,
)

Found 37964 validated image filenames belonging to 1 classes.


In [7]:
with tf.device(f'/device:GPU:{GPU}'):
    model = tf.saved_model.load(f'checkpoints/mobilenetv2/')

In [8]:
img_paths = df['id']

In [9]:
results = []
for path in tqdm(img_paths):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img = cv2.resize(img, (size, size))
    img /= 255.
    result = model(img[np.newaxis,...])
    c = np.argmax(result, axis=1)[0]
    p = np.max(result)
    results.append([c, p])

100%|██████████| 37964/37964 [07:37<00:00, 82.90it/s]


In [10]:
results

[[727, 0.030100528],
 [273, 0.06539173],
 [727, 0.046904173],
 [851, 0.029915906],
 [273, 0.10736413],
 [462, 0.070964865],
 [273, 0.08533058],
 [245, 0.086761385],
 [245, 0.03420757],
 [245, 0.050776895],
 [245, 0.075574316],
 [245, 0.051993303],
 [245, 0.04425842],
 [245, 0.09079704],
 [245, 0.10881834],
 [273, 0.0794625],
 [273, 0.063375704],
 [245, 0.08225106],
 [245, 0.08524639],
 [245, 0.02427073],
 [273, 0.05870842],
 [245, 0.047686063],
 [245, 0.047740646],
 [245, 0.02742428],
 [245, 0.07413754],
 [245, 0.05857488],
 [245, 0.09883549],
 [245, 0.13663143],
 [245, 0.06527302],
 [115, 0.08244092],
 [245, 0.047046974],
 [245, 0.0695053],
 [245, 0.07705793],
 [245, 0.08236722],
 [245, 0.100654416],
 [245, 0.08938103],
 [245, 0.07696933],
 [245, 0.07612492],
 [245, 0.06662238],
 [245, 0.056091316],
 [727, 0.046904173],
 [273, 0.15259881],
 [273, 0.08800568],
 [273, 0.03863656],
 [273, 0.07539269],
 [273, 0.08533058],
 [273, 0.080020554],
 [245, 0.09156421],
 [245, 0.05912243],
 [245,

In [12]:
result_df = pd.DataFrame(results, columns=['landmark_id', 'conf'])
result_df.head()

,landmark_id,conf
0,727,0.030101
1,273,0.065392
2,727,0.046904
3,851,0.029916
4,273,0.107364


In [13]:
DATA_PATH = f'../../Datasets/public/'
TRAIN = DATA_PATH + f'train/'
TEST = DATA_PATH + f'test/'

In [18]:
submission = pd.read_csv(DATA_PATH+f'sample_submisstion.csv')
submission = submission['id']
submission.head()

0    xlf1tgh2ih
1    68a3ot4osk
2    si2lek4u0a
3    rmtqxhipnv
4    2flmjdud0e
Name: id, dtype: object

In [21]:
final = pd.concat([submission, result_df], axis=1)
final.head()

,id,landmark_id,conf
0,xlf1tgh2ih,727,0.030101
1,68a3ot4osk,273,0.065392
2,si2lek4u0a,727,0.046904
3,rmtqxhipnv,851,0.029916
4,2flmjdud0e,273,0.107364


In [23]:
final.to_csv('sample_submisstion.csv', index=False)

In [24]:
pd.read_csv('sample_submisstion.csv')

,id,landmark_id,conf
0,xlf1tgh2ih,727,0.030101
1,68a3ot4osk,273,0.065392
2,si2lek4u0a,727,0.046904
3,rmtqxhipnv,851,0.029916
4,2flmjdud0e,273,0.107364
...,...,...,...
37959,8nlfrrdnwk,351,0.037073
37960,k0w00aa3iy,73,0.042410
37961,xrp8d0pb85,695,0.147802
37962,uobnsz7na9,351,0.103978
